# Before going to the main content ...
let's check if nvcc is available.<br>
If it shows a report on the Cuda compiler driver, then we are good to go!

In [7]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Thu_Nov_18_09:45:30_PST_2021
Cuda compilation tools, release 11.5, V11.5.119
Build cuda_11.5.r11.5/compiler.30672275_0


# Hello world!

Let's try to run the hello-world program.<br>
Let me explain the code first.<br>
The code is located in this directory : ```codes/1_introduction/```<br>
It is a ```.cu``` file, which looks very similar to a ```.c``` file.<br>
However, some necessary changes have to be made.
- The function that runs in GPU is a ```__global__ void``` type.
- GPU functions ALWAYS have to be ```void``` type. (please confirm this!)
- The syntax for calling the GPU function is : ```func <<<nbatch, nthread>>>();```
- The output of the GPU is collected after synchronizing with the CPU by doing ```cudaDeviceSynchronize();```.<br>

The syntax for compiling and running is the following. <br>
```nvcc -o output_exe input_code -run```<br>

In this particular example, there are three parts.<br>

#### A function that runs on the CPU:
```
void helloCPU(){
    printf("Hello from the CPU.\n");
}
```
#### A function that runs on the GPU:

```
__global__ void helloGPU(){
    printf("Hello from the GPU.\n");
}
```
#### A main function:
```
int main(){
    helloCPU();
    helloGPU<<<1, 1>>>();
    cudaDeviceSynchronize();
}
```
Inside the main function, after compilation, the CPU runs the ```helloCPU()``` function just like a regular C program. Then the ```helloGPU()``` function is sent to block=1 and thread=1 of the GPU. So this only runs once. After that, ```cudaDeviceSynchronize()``` makes sure that the GPU and the CPU are synchronized. Now, the output from the GPU is available to the CPU, so that "Hello from the GPU" can be printed on the screen.

In [8]:
!nvcc -o executables/hello codes/1_introduction/hello.cu -run

Hello from the CPU.
Hello from the GPU.
